In [1]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
import json
from pandas.io.json import json_normalize

%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\akkrishn\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
value = input("Enter Stock Symbol:")

Enter Stock Symbol: gme


In [3]:
# Read your api key environment variable
from newsapi import NewsApiClient
load_dotenv()
api_key = os.getenv("NEWSAPI_KEY")

In [4]:
# Create a newsapi client
newsapi = NewsApiClient(api_key=api_key)

In [12]:
# Fetch the Bitcoin news articles
stock_headlines = newsapi.get_everything(q=value,
                                        language="en",
                                        page_size=100,
                                        sort_by='relevancy')
#save json
with open('stock.json', 'w') as json_file:
   json.dump(stock_headlines, json_file)

In [13]:
# Fetch the Bitcoin news articles
fed_headlines = newsapi.get_everything(q='fed',
                                        language="en",
                                        page_size=100,
                                        sort_by='relevancy')
#save json
with open('fed.json', 'w') as json_file:
   json.dump(fed_headlines, json_file)

In [14]:
# Create the Bitcoin sentiment scores DataFrame
tc = pd.read_json ('stock.json')
#parse the json and load into a dataframe
stock_df = json_normalize(stock_headlines['articles'])
#drop all columns except for description
stock_df = stock_df.drop(['author', 'title', 'url','urlToImage','publishedAt','content','source.id','source.name'], axis=1)
#instantiate vader sentiment 
sentiment_title = stock_df.apply(lambda x : analyzer.polarity_scores(x['description']), axis=1)
#add a column for each sentimement metric
stock_df['Comp'] =sentiment_title.apply(lambda x : x.get('compound') )
stock_df['Pos'] =sentiment_title.apply(lambda x : x.get('pos') )
stock_df['Neu'] =sentiment_title.apply(lambda x : x.get('neu') )
stock_df['Neg'] =sentiment_title.apply(lambda x : x.get('neg') )
stock_df

<ipython-input-14-192e310283a8>:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  stock_df = json_normalize(stock_headlines['articles'])


,description,Comp,Pos,Neu,Neg
0,The biggest disappointment was its lack of gui...,-0.6808,0.000,0.663,0.337
1,It looks like investors of GameStop (NYSE: GME...,0.3612,0.088,0.912,0.000
2,GameStop (GME) doesn't possess the right combi...,0.2263,0.076,0.924,0.000
3,The biggest disappointment was its lack of gui...,0.1280,0.199,0.619,0.182
4,"To bolster e-commerce operations, GameStop (GM...",0.0000,0.000,1.000,0.000
...,...,...,...,...,...
95,"This week: Apple’s App Store is dealt a blow, ...",0.0000,0.000,1.000,0.000
96,GameStop (GME) trades with a market cap larger...,0.0000,0.000,1.000,0.000
97,It was a mixed week for meme stocks.AMC (AMC) ...,0.4588,0.100,0.900,0.000
98,"Roger Fan, chief investment officer of RF Capi...",0.3182,0.076,0.924,0.000


In [15]:
# Create the Bitcoin sentiment scores DataFrame
tc = pd.read_json ('fed.json')
#parse the json and load into a dataframe
fed_df = json_normalize(stock_headlines['articles'])
#drop all columns except for description
fed_df = fed_df.drop(['author', 'title', 'url','urlToImage','publishedAt','content','source.id','source.name'], axis=1)
#instantiate vader sentiment 
sentiment_title = fed_df.apply(lambda x : analyzer.polarity_scores(x['description']), axis=1)
#add a column for each sentimement metric
fed_df['Comp'] =sentiment_title.apply(lambda x : x.get('compound') )
fed_df['Pos'] =sentiment_title.apply(lambda x : x.get('pos') )
fed_df['Neu'] =sentiment_title.apply(lambda x : x.get('neu') )
fed_df['Neg'] =sentiment_title.apply(lambda x : x.get('neg') )
fed_df

<ipython-input-15-b93e20203142>:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  fed_df = json_normalize(stock_headlines['articles'])


,description,Comp,Pos,Neu,Neg
0,The biggest disappointment was its lack of gui...,-0.6808,0.000,0.663,0.337
1,It looks like investors of GameStop (NYSE: GME...,0.3612,0.088,0.912,0.000
2,GameStop (GME) doesn't possess the right combi...,0.2263,0.076,0.924,0.000
3,The biggest disappointment was its lack of gui...,0.1280,0.199,0.619,0.182
4,"To bolster e-commerce operations, GameStop (GM...",0.0000,0.000,1.000,0.000
...,...,...,...,...,...
95,"This week: Apple’s App Store is dealt a blow, ...",0.0000,0.000,1.000,0.000
96,GameStop (GME) trades with a market cap larger...,0.0000,0.000,1.000,0.000
97,It was a mixed week for meme stocks.AMC (AMC) ...,0.4588,0.100,0.900,0.000
98,"Roger Fan, chief investment officer of RF Capi...",0.3182,0.076,0.924,0.000
